# Import Image Question

In [1]:
question='3.png'

## Remove Yellow Blue

In [2]:
import cv2
import numpy as np
from PIL import Image
# Load the no_yellow_blue_image
no_yellow_blue_image = cv2.imread(question)

# Convert no_yellow_blue_image to HSV color space
hsv = cv2.cvtColor(no_yellow_blue_image, cv2.COLOR_BGR2HSV)

# Define range of yellow color in HSV
lower_yellow = np.array([20, 100, 100])
upper_yellow = np.array([30, 255, 255])

# Define range of blue color in HSV
lower_blue = np.array([90, 50, 50])
upper_blue = np.array([130, 255, 255])

# Threshold the HSV no_yellow_blue_image to get only yellow and blue colors
yellow_mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)

# Combine masks
mask = yellow_mask + blue_mask

# Morphological operations
kernel = np.ones((5,5), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

# Replace yellow and blue regions with white color
no_yellow_blue_image[np.where(mask==255)] = [255, 255, 255]

#convert image to gray
gray_image = cv2.cvtColor(no_yellow_blue_image, cv2.COLOR_BGR2GRAY)  # If the array is in BGR format

cv2.imwrite('no_yellow_blue_grey_image.png', gray_image)


True

## Get Straight lines image

In [3]:
import cv2
import numpy as np

# Load the grayscale image
image = cv2.imread('no_yellow_blue_grey_image.png', cv2.IMREAD_GRAYSCALE)

# Define a kernel for morphological operations
kernel = np.ones((5, 5), np.uint8)

# Perform morphological opening (erosion followed by dilation) to remove noise
opening = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

# Perform morphological closing (dilation followed by erosion) to fill in gaps
morph_image = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)


In [4]:
x=[]
for i in morph_image:
    temp=[]
    for j in i:
        if(250<=j and j>=255):
            temp.append(255)
            # temp.append(0)
        else:
            temp.append(0)
    x.append(temp)    

b_w_image_array = np.array(x)
b_w_image = Image.fromarray(b_w_image_array.astype(np.uint8))

In [5]:
b_w_image.save('perfect_b_w_image.png')

## Get start and end node

In [6]:
import cv2
import numpy as np


# Read the binary image
binary_image = cv2.imread('perfect_b_w_image.png', cv2.IMREAD_GRAYSCALE)

# Find contours in the binary image
contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Sort contours by area in descending order
contours = sorted(contours, key=cv2.contourArea, reverse=True)

# Assume the outermost contour is the maze boundary
maze_boundary = contours[0]

# Find the bounding rectangle of the maze boundary
x, y, w, h = cv2.boundingRect(maze_boundary)

# Load the original image
original_image = cv2.imread(question)

# Define color thresholds for yellow and blue
lower_yellow = np.array([20, 100, 100])
upper_yellow = np.array([30, 255, 255])
lower_blue = np.array([110, 50, 50])
upper_blue = np.array([130, 255, 255])

# Convert the image to HSV color space
hsv_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2HSV)

# Mask for yellow and blue regions
yellow_mask = cv2.inRange(hsv_image, lower_yellow, upper_yellow)
blue_mask = cv2.inRange(hsv_image, lower_blue, upper_blue)

# Find contours in the yellow and blue masks
yellow_contours, _ = cv2.findContours(yellow_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
blue_contours, _ = cv2.findContours(blue_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Find the extreme left point of the yellow region
if yellow_contours:
    leftmost_yellow = tuple(yellow_contours[0][yellow_contours[0][:, :, 0].argmin()][0])
    yellow_start_point = (leftmost_yellow[0], leftmost_yellow[1])
else:
    yellow_start_point = None

# Find the extreme right point of the blue region
if blue_contours:
    rightmost_blue = tuple(blue_contours[0][blue_contours[0][:, :, 0].argmax()][0])
    blue_end_point = (rightmost_blue[0], rightmost_blue[1])
else:
    blue_end_point = None


# Find the extreme top point of the yellow region
start_point = tuple(yellow_contours[0][yellow_contours[0][:, :, 1].argmin()][0]) if yellow_contours else None

# Find the extreme right point of the blue region
end_point = tuple(blue_contours[0][blue_contours[0][:, :, 0].argmax()][0]) if blue_contours else None



## Add nodes on image

In [7]:
import cv2
import numpy as np

def mark_corners(img, corners):
    for x, y in corners:
        img[x, y, :] = [0, 0, 255]  # Set the pixel color at the corner location to red
        # cv2.circle(img, (y, x), 3, [0, 0, 255], -1)


def color_pixels_same_x(img, corner_locations):
    unique_x_coordinates = np.unique(corner_locations[:, 1])

    for x_coord in unique_x_coordinates:
        pixels_with_same_x = corner_locations[corner_locations[:, 1] == x_coord]
        for x, y in pixels_with_same_x:
            # img[x, y] = [0, 0, 255]  # Set the pixel color to red
            img[x, y] = 50  # Set the pixel color to red
            


def process_image(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sharpening_filter = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    sharpened_img = cv2.filter2D(gray, -1, sharpening_filter)
    corners = cv2.cornerHarris(sharpened_img, blockSize=4, ksize=1, k=0.05)
    corner_locations = np.column_stack(np.where(corners > 0.01 * corners.max()))
    mark_corners(img, corner_locations)
    return corner_locations


def print_adjacency_list(corner_locations):
    num_corners = len(corner_locations)
    adjacency_list = {i: [] for i in range(num_corners)}

    for i in range(num_corners):
        for j in range(i + 1, num_corners):
            distance = np.linalg.norm(np.array(corner_locations[i]) - np.array(corner_locations[j]))
            if distance < 20:
                adjacency_list[i].append(j)
                adjacency_list[j].append(i)

    print("Adjacency List:")
    for node, neighbors in adjacency_list.items():
        print(f"{node}: {neighbors}")


perfect_b_w_with_nodes_image = cv2.imread("perfect_b_w_image.png")
corner_locations = process_image(perfect_b_w_with_nodes_image)
color_pixels_same_x(perfect_b_w_with_nodes_image, corner_locations)
cv2.imwrite("perfect_b_w_with_nodes_image.png", perfect_b_w_with_nodes_image)



True

In [8]:
# Draw the start and end points on the original image
perfect_b_w_with_nodes_image = cv2.imread("perfect_b_w_with_nodes_image.png",cv2.IMREAD_GRAYSCALE)
if start_point:
    # x=start_point[1]
    # y=start_point[0]
    # box_size = 5
    # box_half = box_size // 2

    # for i in range(x - box_half, x + box_half + 1):
    #     for j in range(y - box_half, y + box_half + 1):
    #         if 0 <= i < perfect_b_w_with_nodes_image.shape[0] and 0 <= j < perfect_b_w_with_nodes_image.shape[1]:
    #             perfect_b_w_with_nodes_image[i][j] = 50
    cv2.circle(perfect_b_w_with_nodes_image, start_point, 4, 50, -1)  # Green for start point
if end_point:
    perfect_b_w_with_nodes_image[end_point[1], end_point[0]] = 50


cv2.imwrite('maze_with_all_nodes.png', perfect_b_w_with_nodes_image)



True

In [9]:
final_image=perfect_b_w_with_nodes_image

# Create Graph

In [10]:
from src.maze import MazeGraph
# Create a new graph
graph = MazeGraph()

In [11]:
def get_length(node_a, node_b):
    return np.linalg.norm(np.array(graph.get_nodes(node_a)) - np.array(graph.get_nodes(node_b)))

## Go Horizontal


In [12]:
for y in range(0,final_image.shape[0]):
    # node_list
    node_list=[]
    for x in range(0,final_image.shape[1]):
        if(final_image[y][x]==50):
            if(len(node_list)==0):
                node_list.append(graph.add_node((x,y)))
            else:
                node_list.append(graph.add_node((x,y)))
                graph.add_edge(node_list[0],node_list[1],length=get_length(node_list[0],node_list[1]))
                node_list.pop(0)

        elif(final_image[y][x]==0):
            node_list=[]

## Go Vertical

In [13]:
for x in range(0,final_image.shape[1]):
    # node_list
    node_list=[]
    for y in range(0,final_image.shape[0]):
        if(final_image[y][x]==50):
            if(len(node_list)==0):
                    node_list.append(graph.get_node_from_coordinates((x,y)))
            else:
                node_list.append(graph.get_node_from_coordinates((x,y)))
                graph.add_edge(node_list[0],node_list[1],length=get_length(node_list[0],node_list[1]))
                node_list.pop(0)

        elif(final_image[y][x]==0 and final_image[y-1][x]!=255):
            node_list=[]

In [14]:
graph.set_start_node(graph.get_node_from_coordinates(start_point))
graph.set_end_node(graph.get_node_from_coordinates(end_point))


In [15]:
from src.pathfinder import PathFinder
x=PathFinder(maze_graph=graph)
solution=x.modified_a_star()

In [16]:
answer=cv2.imread(question)
for i in solution:
    x=graph.get_nodes(i)
    answer[x[1],x[0]]=(0,0,255)


In [17]:
import cv2
import math

# Function to calculate angle between three points
def calculate_angle(pt1, pt2, pt3):
    angle_radians = math.atan2(pt3[1] - pt2[1], pt3[0] - pt2[0]) - math.atan2(pt1[1] - pt2[1], pt1[0] - pt2[0])
    angle_degrees = math.degrees(angle_radians)
    return angle_degrees

# Function to get direction from angle
def get_direction(angle):
    if angle < -75:
        return "Right"
    elif angle > 75:
        return "Left"
    else:
        return "Straight"

# Function to draw lines between nodes
def draw_lines(image, nodes):
    turns = []
    num_turns = 0
    prev_direction = None
    for i in range(len(nodes) - 2):
        node1 = nodes[i]
        node2 = nodes[i + 1]
        node3 = nodes[i + 2]
        cv2.line(image, node1, node2, (0, 0, 0), 3)  # Draw line between the nodes
        angle = calculate_angle(node1, node2, node3)
        direction = get_direction(angle)
        if abs(angle) > 89 and direction != "Straight":
            if prev_direction != direction:
                turns.append((node2, direction))
                num_turns += 1
            prev_direction = direction
    return turns, num_turns

# Example solutions list
sol = solution  # Assuming you have a list of node numbers

# List to store the coordinates of nodes in the solution
nodes = []

# Retrieve coordinates of nodes in the solution
for i in sol:
    x, y = graph.get_nodes(i)  # Assuming graph.get_nodes() returns (x, y) coordinates
    nodes.append((x, y))  # Append coordinates to the list

image = cv2.imread(question) 

# Draw lines between nodes in the solution on the image and get the turns
turns, num_turns = draw_lines(image, nodes)

# Show the image with the colored pixel
cv2.imshow('Connected Nodes Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Write turns to a text file
with open("turns.txt", "w") as file:
    file.write("Number of turns: " + str(num_turns) + "\n")
    file.write("Turns:\n")
    for turn in turns:
        file.write("At point " + str(turn[0]) + " turn " + turn[1] + "\n")
